In [1]:
import requests
token = "apikey " + "NkgYO9eRx11Hd5YaDPIfr8W00knPdKKegHTJ"

headers = {"Authorization":token}
#Transport for NSW Open data
base_url = "https://api.transport.nsw.gov.au/v1/gtfs/vehiclepos/sydneytrains"
train = requests.get(base_url, headers= headers)


In [2]:
#gtfs-realtime-bindings: for handling the GTFS-realtime format
#protobuf: for handling Protocol Buffers generally

from google.transit import gtfs_realtime_pb2

data = gtfs_realtime_pb2.FeedMessage()

data.ParseFromString(train.content)


35401

In [16]:
for entity in data.entity[:5]:
    print(entity.vehicle.trip.route_id,
          entity.vehicle.trip.schedule_relationship,
          entity.vehicle.position.latitude,
          entity.vehicle.position.longitude,
          entity.vehicle.vehicle.label 
         )

BNK_2a 0 -33.89546203613281 151.19009399414062 16:14 Redfern Station to Central Station 
APS_1a 0 -34.02763366699219 150.84242248535156 13:57 Central Station to Campbelltown Station 
NSN_2a 0 -33.86575698852539 151.20556640625 15:05 North Sydney Station to Central Station 
NSN_2a 0 -33.84431076049805 151.2112579345703 15:04 Lindfield Station to Central Station 
WST_1a 0 -33.889034271240234 151.20118713378906 15:26 Central Station to Emu Plains Station 


In [10]:
import csv
location_output = [] # a list of lists for train position data

#Each train's key data into the list
for entity in data.entity:
    location_output.append([entity.id,
                             entity.vehicle.trip.route_id, 
                             entity.vehicle.trip.trip_id,
                             entity.vehicle.trip.schedule_relationship,
                             entity.vehicle.position.latitude, 
                             entity.vehicle.position.longitude,
                             entity.vehicle.congestion_level,
                             entity.vehicle.stop_id,
                             entity.vehicle.vehicle.id,
                             entity.vehicle.vehicle.label 
                            ])

# write the bus position data to the positions.csv
with open("realtime-train.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(['id','route_id','trip_id','schedule_relationship','latitude','longitude','congestion_level','stop_id', 'vehicle_id','label'])
    writer.writerows(location_output)

In [11]:
import pandas as pd
realtime_train = pd.read_csv('realtime-train.csv')

In [12]:
realtime_train.head()

,id,route_id,trip_id,schedule_relationship,latitude,longitude,congestion_level,stop_id,vehicle_id,label
0,1,BNK_2a,1--J.1352.124.64.M.8.55912374,0,-33.895462,151.190094,0,Sydney.SY511 Loc,5818.7402.3198.8964.8873.4661.4268.9755,16:14 Redfern Station to Central Station
1,2,APS_1a,10-K.1352.124.124.M.8.55910687,0,-34.027634,150.842422,0,Glenfield.Minto 2 Loc,7243.3876.9289.1982.6916.1447.7343.2206,13:57 Central Station to Campbelltown Station
2,3,NSN_2a,101L.1352.124.64.T.8.55910501,0,-33.865757,151.205566,0,Sydney.Wynyard 3 Loc,3881.5156.4759.4561.5012.4370.1557.2369,15:05 North Sydney Station to Central Station
3,4,NSN_2a,102L.1352.124.64.T.8.55910308,0,-33.844311,151.211258,0,NorthSydney.SH2.90 Loc,9962.1315.5042.5711.1340.2496.5468.6145,15:04 Lindfield Station to Central Station
4,5,WST_1a,104M.1352.124.64.T.8.55910764,0,-33.889034,151.201187,0,Sydney.SY437 Loc,6929.4778.7610.9879.9745.8843.2555.2527,15:26 Central Station to Emu Plains Station


In [13]:
realtime_train.shape

(197, 10)

In [14]:
realtime_train['schedule_relationship'].value_counts()

0    197
Name: schedule_relationship, dtype: int64

In [17]:
import folium

In [20]:
lat=-33.87
lng=151.1

sydney_map = folium.Map(location=[lat, lng], zoom_start=12)

In [21]:
sydney_map

In [26]:
locations = folium.map.FeatureGroup()

for lat, lng, in zip(realtime_train.latitude, realtime_train.longitude):
    locations.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='red',
            fill_color='green',
            fill_opacity=0.6
        )
    )

In [27]:
sydney_map.add_child(locations)